<a id='index'></a>

# 🍏 quick clean up of old notebook with image/metadata model 🍏

In [ ]:
import os
import sys

sys.path.insert(0, "/projects/bcrv/abrown3/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns
import random

from tqdm.auto import tqdm

import torch
import math
from datetime import datetime

import optuna
from optuna.exceptions import DuplicatedStudyError
import torch

In [ ]:
from pathlib import Path
from types import SimpleNamespace
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.nn.utils.rnn import pad_sequence

from sklearn.preprocessing import label_binarize
from sklearn.metrics import (
    average_precision_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    fbeta_score,
    top_k_accuracy_score,
    roc_curve,
    precision_recall_curve,
    confusion_matrix,
    classification_report,
)
from matplotlib.ticker import AutoMinorLocator
from matplotlib import cm as _cm
from collections import Counter

In [ ]:
import torch.nn as nn
import torch.optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau, LinearLR
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset

In [ ]:
# for the pandas warnings i will continue to ignore until i can't any longer
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

# Part 1: dataset, dataloader

we will use the following config file (which will eventually be cleaned up) for basically everything......... 
- if you keep `'use_wandb': True,`, and use the current project please add tags `config['wandb_tags']` or `config['wandb_notes']` to say who is running the model and when! i beg of you!
    - ex tags: I use `'wandb_tags'`: `'Delta!'` for every run that is done on Delta, other tags depnd on my mood. other useful ones are `'wandb_tags'`:`group labels`, `group SN` for class info that is easily visible without scrolling the through columns on the project page

In [ ]:
config = {
    "project": "AppleCiDEr",
    "config_from": None,
    "random_seed": 42,  # 42, 66, 0, 12, 123
    "use_wandb": False,
    "save_weights": False,
    "weights_path": f'/projects/bcrv/abrown3/AppleCiDEr_Skyportal/cider_weights/multi-extra-{datetime.now().strftime("%Y-%m-%d-%H-%M")}',
    "use_pretrain": None,
    "freeze": False,
    "mode": "all",  # 'photo', 'spectra', ;metdata & images', 'all', 'ztf'
    ## Data General
    "step": "type",
    #'classes': ['SN I','SN II', 'Cataclysmic', 'AGN', 'Tidal Disruption Event'],
    "classes": [
        "SN Ia",
        "SN Ib",
        "SN Ic",
        "SN II",
        "SN IIP",
        "SN IIn",
        "Cataclysmic",
        "AGN",
        "Tidal Disruption Event",
    ],
    "group_labels": True,
    "num_classes": 10,
    #'max_samples': 5500,
    "gpu": 1,
    ## Data General
    "preprocessed_path": "/work/nvme/bcrv/abrown3/preprocessed_data/data_multi/day10",
    # for ALERTS
    "train_csv_path": "/projects/bcrv/abrown3/AppleCiDEr_csv/AppleCider_Train_vetted_7-6.csv",
    "val_csv_path": "/projects/bcrv/abrown3/AppleCiDEr_csv/AppleCider_Val_vetted_7-6.csv",
    "test_csv_path": "/projects/bcrv/abrown3/AppleCiDEr_csv/AppleCider_Test_vetted_7-6.csv",
    # for SPECTRA
    "spec_dir": "/work/nvme/bcrv/mxu11",
    ## personal tag for weights files in local folder
    "custom_weight_path": False,
    "custom_weight_name": "-",
    ## only for the wandb users.....
    "use_notes_tags": False,
    "wandb_tags": ["Delta!", "multi"],
    "wandb_notes": f"test",
    ## Photometry Model 🍏🍏
    "photo_event_path": "/work/hdd/bcrv/ffontinelenunes/data/AppleCider/photo_events",
    "output_dir": "/work/hdd/bcrv/ffontinelenunes/data/AppleCider/photo_events",
    "stats_file": "/work/hdd/bcrv/ffontinelenunes/data/AppleCider/photo_events/feature_stats_day100.npz",
    "horizon_days": 10.0,  # <- fine-tuning on 50 days
    "sampler_balance": False,
    "num_workers": 0,
    "p_d_model": 128,
    "p_n_heads": 8,  # 4,
    "p_n_layers": 4,  # 2,
    "p_dropout": 0.30,
    "max_len": 257,  # 300,#256,#128,#128,
    #'lr' : 5e-6,
    # 'weight_decay' : 1e-2,
    # 'focal_gamma':2.0,
    #'cut_time_p':None, #(.25,.25,.25,.25), #None,  # or (.25,.25,.25,.25)
    #'p_dropout':0.1,
    #'jitter_scale':0.10,
    #'flux_nu':8,
    # training schedule
    #'epochs' :150,
    #'patience' :30,
    # misc
    #'seed':42,
    #'NUM_CLASSES':5,
    ## Spectra Model 🍏🍏🍏🍏🍏
    # "learning_rate",
    # "weight_decay": 1e-5,
    # "ema_decay": 0.995,
    # "focal_loss_gamma": 2.0,
    # "warmup_epochs": 10,
    "T_0": 5,
    "T_mult": 1,
    "eta_min": 1e-5,
    "start_factor": 1e-6,
    "end_factor": 1.0,
    # "num_workers": 0,
    # "sampling": False,
    # "epochs": 100,
    # "optimizer": "AdamW",
    # "patience": 14,
    "train_dir": "/work/nvme/bcrv/mxu11",
    "val_dir": "/work/nvme/bcrv/mxu11",
    "test_dir": "/work/nvme/bcrv/mxu11",
    "class_order": ["SN I", "SN II", "Cataclysmic", "AGN", "Tidal Disruption Event"],
    ## Metadata & Image Model 🍏🍏
    "num_experts": 4,
    "towers_hidden_dims": 16,
    "towers_outdims": 4,
    "embedding": "False",
    "fusion_hidden_dims": 128,
    "fusion_router_dims": 128,
    "fusion_outdims": 4,
    "cnn_lr": 2,
    "cnn_decay": 5e-2,
    "psf_lr": 0.5,
    "psf_decay": 5e-2,
    "mag_lr": 2,
    "mag_decay": 0.0,
    "lc_lr": 2,
    "lc_decay": 0.05,
    "spatial_lr": 2,
    "spatial_decay": 0.0,
    "coord_lr": 0.5,
    "coord_decay": 0.0,
    "nst1_lr": 2,
    "nst1_decay": 0.0,
    "nst2_lr": 2,
    "nst2_decay": 0.0,
    "fusion_lr": 1,
    "fusion_decay": 1e-2,
    "fusion_beta1": 0.9,
    "fusion_beta2": 0.999,
    "router_decay": 0.0,
    "router_lr": 1.5,
    "router_beta1": 0.9,
    "router_beta2": 0.999,
    "router_lr_2": 1,
    "router_beta1_2": 0.95,
    "router_beta2_2": 0.99,
    "classifier_decay": 0.0,
    "classifier_lr": 2.44,
    "classifier_beta1": 0.95,
    "classifier_beta2": 0.99,
    "beta1": 0.9,
    "beta2": 0.999,
    "eps": 5e-10,
    "sched_pat": 5,
    "sched_factor": 0.4,
    "min_lr": 6e-10,
    "weight_exp": 1,
    "gamma": 2.5,
    "criterion": "cross_entropy",
    "scheduler": "cosine_annealing",
    "t_max": 6,
    "max_norm": 5,
    ## MultiModal Model 🍏🍏
    "hidden_dim": 64,
    "fusion": "avg",  # 'avg', 'concat'
    ## Training
    "batch_size": 32,
    "lr": 0.001,
    "beta1": 0.9,
    "beta2": 0.999,
    "weight_decay": 0.01,
    "epochs": 5,
    "early_stopping_patience": 4,
    "scheduler": "ReduceLROnPlateau",  # 'ExponentialLR', 'ReduceLROnPlateau'
    "gamma": 0.9,  # for ExponentialLR scheduler
    "factor": 0.3,  # for ReduceLROnPlateau scheduler
    "patience": 3,  # for ReduceLROnPlateau scheduler
    "warmup": False,
    "warmup_epochs": 10,
}

In [ ]:
def get_config(trial):
    config = {
        "project": "AppleCiDEr",
        "config_from": None,
        "random_seed": 42,  # 42, 66, 0, 12, 123
        "use_wandb": False,
        "save_weights": False,
        "weights_path": f'/projects/bcrv/abrown3/AppleCiDEr_Skyportal/cider_weights/multi-extra-{datetime.now().strftime("%Y-%m-%d-%H-%M")}',
        "use_pretrain": None,
        "freeze": False,
        "step": "type",
        "classes": [
            "SN Ia",
            "SN Ib",
            "SN Ic",
            "SN II",
            "SN IIP",
            "SN IIn",
            "Cataclysmic",
            "AGN",
            "Tidal Disruption Event",
        ],
        "group_labels": True,
        "num_classes": 10,
        #'max_samples': 5500,
        ## Data General
        "preprocessed_path": "/work/nvme/bcrv/abrown3/preprocessed_data/data_multi/day10",
        # for ALERTS
        "train_csv_path": "/projects/bcrv/abrown3/AppleCiDEr_csv/AppleCider_Train_vetted_7-6.csv",
        "val_csv_path": "/projects/bcrv/abrown3/AppleCiDEr_csv/AppleCider_Val_vetted_7-6.csv",
        "test_csv_path": "/projects/bcrv/abrown3/AppleCiDEr_csv/AppleCider_Test_vetted_7-6.csv",
        # for SPECTRA
        "spec_dir": "/work/nvme/bcrv/mxu11",
        #'class_weights': False,
        #'generate_train_val_files': False,
        ## personal tag for weights files in local folder
        "custom_weight_path": False,
        "custom_weight_name": "-",
        ## only for the wandb users.....
        "use_notes_tags": False,
        "wandb_tags": ["Delta!"],
        "wandb_notes": f" 9 class version",
        ## Photometry Model 🍏🍏
        "photo_event_path": "/work/hdd/bcrv/ffontinelenunes/data/AppleCider/photo_events",
        "output_dir": "/work/hdd/bcrv/ffontinelenunes/data/AppleCider/photo_events",
        "stats_file": "/work/hdd/bcrv/ffontinelenunes/data/AppleCider/photo_events/feature_stats_day100.npz",
        "horizon_days": 10.0,  # <- fine-tuning on 50 days
        #'batch_size'   :  256, #64,
        "sampler_balance": False,
        "num_workers": 0,
        # model stuff
        "p_d_model": 128,
        "p_n_heads": 8,  # 4,
        "p_n_layers": 4,  # 2,
        "p_dropout": 0.30,
        "max_len": 257,  # 300,#256,#128,#128,
        #'lr' : 5e-6,
        # 'weight_decay' : 1e-2,
        # 'focal_gamma':2.0,
        #'cut_time_p':None, #(.25,.25,.25,.25), #None,  # or (.25,.25,.25,.25)
        #'p_dropout':0.1,
        #'jitter_scale':0.10,
        #'flux_nu':8,
        # training schedule
        #'epochs' :150,
        #'patience' :30,
        # misc
        #'seed':42,
        #'NUM_CLASSES':5,
        ## Spectra Model 🍏🍏
        # "learning_rate",
        # "weight_decay": 1e-5,
        # "ema_decay": 0.995,
        # "focal_loss_gamma": 2.0,
        # "warmup_epochs": 10,
        "T_0": 5,
        "T_mult": 1,
        "eta_min": 1e-5,
        "start_factor": 1e-6,
        "end_factor": 1.0,
        # "num_workers": 0,
        # "sampling": False,
        # "epochs": 100,
        # "optimizer": "AdamW",
        # "patience": 14,
        "train_dir": "/work/nvme/bcrv/mxu11",
        "val_dir": "/work/nvme/bcrv/mxu11",
        "test_dir": "/work/nvme/bcrv/mxu11",
        # "class_order": ['SN I', 'SN II', 'Cataclysmic', 'AGN', 'Tidal Disruption Event'],
        "class_order": [
            "SN Ia",
            "SN Ib",
            "SN Ic",
            "SN II",
            "SN IIP",
            "SN IIn",
            "Cataclysmic",
            "AGN",
            "Tidal Disruption Event",
        ],
        #'s_dropout': 0.2,
        #'s_conv_channels': [1, 64, 64, 32, 32],
        #'s_kernel_size': 3,
        #'s_mp_kernel_size': 4,
        ## Metadata & Image Model 🍏🍏
        "num_experts": 4,
        "towers_hidden_dims": 16,
        "towers_outdims": 4,
        "embedding": "False",
        "fusion_hidden_dims": 128,
        "fusion_router_dims": 128,
        "fusion_outdims": 4,
        "cnn_lr": 2,
        "cnn_decay": 5e-2,
        "psf_lr": 0.5,
        "psf_decay": 5e-2,
        "mag_lr": 2,
        "mag_decay": 0.0,
        "lc_lr": 2,
        "lc_decay": 0.05,
        "spatial_lr": 2,
        "spatial_decay": 0.0,
        "coord_lr": 0.5,
        "coord_decay": 0.0,
        "nst1_lr": 2,
        "nst1_decay": 0.0,
        "nst2_lr": 2,
        "nst2_decay": 0.0,
        "fusion_lr": 1,
        "fusion_decay": 1e-2,
        "fusion_beta1": 0.9,
        "fusion_beta2": 0.999,
        "router_decay": 0.0,
        "router_lr": 1.5,
        "router_beta1": 0.9,
        "router_beta2": 0.999,
        "router_lr_2": 1,
        "router_beta1_2": 0.95,
        "router_beta2_2": 0.99,
        "classifier_decay": 0.0,
        "classifier_lr": 2.44,
        "classifier_beta1": 0.95,
        "classifier_beta2": 0.99,
        "beta1": 0.9,
        "beta2": 0.999,
        "eps": 5e-10,
        "sched_pat": 5,
        "sched_factor": 0.4,
        "min_lr": 6e-10,
        "weight_exp": 1,
        "gamma": 2.5,
        "criterion": "cross_entropy",
        "scheduler": "cosine_annealing",
        "t_max": 6,
        "max_norm": 5,
        ## MultiModal Model 🍏🍏
        "hidden_dim": 64,
        "fusion": "avg",  # 'avg', 'concat'
        ## Training
        "batch_size": 32,
        "lr": 0.001,
        "beta1": 0.9,
        "beta2": 0.999,
        "weight_decay": 0.01,
        "epochs": 5,
        "early_stopping_patience": 4,
        "scheduler": "ReduceLROnPlateau",  # 'ExponentialLR', 'ReduceLROnPlateau'
        "gamma": 0.9,  # for ExponentialLR scheduler
        "factor": 0.3,  # for ReduceLROnPlateau scheduler
        "patience": 3,  # for ReduceLROnPlateau scheduler
        "warmup": False,
        "warmup_epochs": 10,
    }

    if STUDY_NAME.startswith("photo"):
        config["mode"] = "photo"
        config["epochs"] = 50
    elif STUDY_NAME.startswith("spectra"):
        config["mode"] = "spectra"
        config["epochs"] = 50
    elif STUDY_NAME.startswith("meta"):
        config["mode"] = "meta"
        config["epochs"] = 50
    elif STUDY_NAME.startswith("image"):
        config["mode"] = "image"
        config["epochs"] = 50

    elif STUDY_NAME.startswith("ztf"):
        config["mode"] = "ztf"
        config["epochs"] = 50

    elif STUDY_NAME.startswith("all"):
        config["mode"] = "all"
        config["epochs"] = 50

    else:
        raise NotImplementedError(f"Unknown study name {STUDY_NAME}")

    if config["mode"] in ("photo", "ztf", "all"):
        config["p_dropout"] = trial.suggest_float("p_dropout", 0.0, 0.4)

    config["lr"] = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    config["factor"] = trial.suggest_float("factor", 0.1, 1.0)
    config["beta1"] = trial.suggest_float("beta1", 0.7, 0.99)
    config["weight_decay"] = trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)

    config["study_name"] = STUDY_NAME

    return config

In [ ]:
Train = pd.read_csv("/projects/bcrv/abrown3/AppleCiDEr_csv/AppleCider_Train_vetted.csv")
# Val = pd.read_csv('/projects/bcrv/abrown3/AppleCiDEr_csv/AppleCider_Val_vetted.csv')
# Test = pd.read_csv('/projects/bcrv/abrown3/AppleCiDEr_csv/AppleCider_Test_vetted.csv')
# Train[:1]

<a id='dataset'></a>
<br>
`CiderDataset`

In [ ]:
class CiderDataset(Dataset):
    def __init__(self, config, split="train"):
        super().__init__()

        self.split = split  #  'train', 'val', 'test'
        self.mode = config["mode"]  #  'all', 'ztf', 'photo', 'metadata & images', 'spectra'
        self.preprocessed_path = config["preprocessed_path"]
        self.step = config["step"]
        self.random_seed = config["random_seed"]  # 42, 66, 0, 12, 123

        # for ALERTS
        self.df_train = config["train_csv_path"]
        self.df_val = config["val_csv_path"]
        self.df_test = config["test_csv_path"]
        # for SPECTRA
        self.spec_dir = config["spec_dir"]

        self.photo_event_path = config[
            "photo_event_path"
        ]  # /work/hdd/bcrv/ffontinelenunes/data/AppleCider/photo_events

        self.group_labels = config["group_labels"]

        if self.split == "train":
            self.df = pd.read_csv(self.df_train)
        elif self.split == "val":
            self.df = pd.read_csv(self.df_val)
        elif self.split == "test":
            self.df = pd.read_csv(self.df_test)
        else:
            raise ValueError("Split must be either train, val, or test.")

        self._split()

        # if self.group_labels:
        #    ## create convenient mapping for label from str to int and from int to str
        #    ## group -> SN I, SN II, CV, AGN, TDE
        #    self.id2target = {0: 'SN I', 1: 'SN II', 2: 'Cataclysmic', 3: 'AGN', 4: 'Tidal Disruption Event'}
        #    self.target2id = {'SN Ia': 0 , 'SN Ic': 0,  'SN Ib': 0, 'SN II': 1, 'SN IIP': 1, 'SN IIn': 1, 'SN IIb': 1, 'Cataclysmic': 2, 'AGN': 3, 'Tidal Disruption Event': 4}

        self.id2target = {
            0: "SN Ia",
            1: "SN Ib",
            2: "SN Ic",
            3: "SN II",
            4: "SN IIP",
            5: "SN IIn",
            6: "Cataclysmic",
            7: "AGN",
            8: "Tidal Disruption Event",
        }
        self.target2id = {v: k for k, v in self.id2target.items()}
        self.num_classes = len(self.target2id)

    def _split(self):
        """sort train, val,test based on df"""
        # this means that you can optionally use multiple alerts or not..
        # doesn't change much except your dataframe would have one row per object

        if self.split == "train":
            self.df = pd.read_csv(self.df_train)
        elif self.split == "val":
            self.df = pd.read_csv(self.df_val)
        elif self.split == "test":
            self.df = pd.read_csv(self.df_test)

        else:
            raise ValueError("Split must be either train, val, or test.")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        el = self.df.iloc[index]  #  get df row for object
        target = self.target2id[el[self.step]]  #  get object type, make str label int

        obj_id, file = el["name"], el["file"]

        file_path = os.path.join(self.preprocessed_path, file)  #  get object alert file name from df
        sample = np.load(file_path, allow_pickle=True).item()  #  load object alert file: .npy

        try:
            # ☆☆☆☆☆☆ Photometry ☆☆☆☆☆☆
            event_path = os.path.join(self.photo_event_path, self.split)
            photo_event = np.load(os.path.join(event_path, f"{obj_id}.npz"))
            photometry = photo_event["data"] if isinstance(photo_event, np.lib.npyio.NpzFile) else photo_event
            event_mjd = el["alert mjd"]

            # new cut to photo event
            photometry = photometry[photometry[:, 0] <= event_mjd]

            _BAND_OH = np.eye(3, dtype=np.float32)
            dt = np.log1p(photometry[:, 0])
            dt_prev = np.log1p(photometry[:, 1])
            logf, logfe = photometry[:, 3], photometry[:, 4]
            oh = _BAND_OH[photometry[:, 2].astype(np.int64)]
            vec4 = np.stack([dt, dt_prev, logf, logfe], 1)
            photo_tensor = torch.from_numpy(np.concatenate([vec4, oh], 1))

        except:
            raise ValueError(f"photo error! at {file}")
        # ☆☆☆☆☆☆ Metadata ☆☆☆☆☆☆
        metadata = sample["metadata norm"]
        metadata_tensor = torch.tensor(metadata)
        metadata_tensor = metadata_tensor.to(torch.float32)

        # ☆☆☆☆☆☆ Images ☆☆☆☆☆☆
        image_tensor = sample["images norm, radial distance"]
        # needs to be converted
        image_tensor = image_tensor.to(torch.float32)

        # ☆☆☆☆☆☆ Spectra ☆☆☆☆☆☆
        obj_spec_file = os.path.join(self.spec_dir, f"{obj_id}.npy")
        flux = np.load(obj_spec_file, allow_pickle=True)
        spectra_tensor = torch.tensor(flux, dtype=torch.float32).unsqueeze(0)

        return photo_tensor, metadata_tensor, image_tensor, spectra_tensor, target

In [ ]:
train_ = CiderDataset(config, split="train")
val_ = CiderDataset(config, split="val")
test_ = CiderDataset(config, split="test")

In [ ]:
photo, metadata, images, spectra, labels = train_[0]

print("photometry: ", photo.shape)
print("metadata: ", metadata.shape)
print("images: ", images.shape)
print("spectra: ", spectra.shape)

<a id='loader'></a>

In [ ]:
photo, photo_mask, metadata, images, spectra, labels = next(iter(train_ld))


print("photometry:      ", photo.shape)
print("photometry mask: ", photo_mask.shape)
print("metadata:        ", metadata.shape)
print("images:          ", images.shape)
print("spectra:         ", spectra.shape)

In [ ]:
print(photo.dtype)
print(photo_mask.dtype)
print(metadata.dtype)
print(images.dtype)
print(spectra.dtype)
print(labels.dtype)

# our models.....

<a id='meta'></a>
### img, meta: XastroMiNN
<br>

<small><i>[back to index](#index)</i></small>

In [ ]:
# horrible import, fix later
from AppleCiDEr_Skyportal.AppleCider.models.XastroMiNN import ResidualTowerBlock, SplitHeadConvNeXt

In [ ]:
device = torch.device("cuda")


class XastroMiNN(nn.Module):
    """
    Image and Metadata transient classifier
    """

    def __init__(
        self,
        num_classes=5,
        num_mlp_experts=4,
        towers_hidden_dims=16,
        towers_outdims=32,
        fusion_hidden_dims=128,
        fusion_router_dims=128,
        fusion_outdims=32,
        config=None,
    ):
        super().__init__()

        # self.classification = True if config['mode'] == 'metdata & images' else False

        self.has_image = True  # Flag for image availability
        self.num_classes = num_classes
        self.towers_hidden_dims = towers_hidden_dims
        self.towers_outdims = towers_outdims

        self.fusion_hidden_dims = fusion_hidden_dims  # was 1024
        self.fusion_router_dims = fusion_router_dims  # was 256
        self.fusion_outdims = fusion_outdims

        # ===== Metadata Processing Towers =====
        # Each tower processes specific metadata features
        # PSF quality features tower
        self.psf_tower = ResidualTowerBlock(2, self.towers_hidden_dims, towers_outdims)

        # Magnitude features tower
        self.mag_tower = ResidualTowerBlock(7, self.towers_hidden_dims * 2, towers_outdims)

        # LC features tower
        self.lc_tower = ResidualTowerBlock(12, self.towers_hidden_dims * 3, towers_outdims)

        # Spatial features tower (distpsnr1, distpsnr2, nmtchps)
        self.spatial_tower = ResidualTowerBlock(3, self.towers_hidden_dims, towers_outdims)

        # Nearest source features tower 1 (sgscore1, distpsnr1)
        self.nst1_tower = ResidualTowerBlock(2, self.towers_hidden_dims, fusion_outdims)
        # self.mega_tower = ResidualTowerBlock(7, 64, self.towers_outdims)
        # Nearest source features tower 2 (sgscore2, distpsnr2)
        self.nst2_tower = ResidualTowerBlock(2, self.towers_hidden_dims, fusion_outdims)

        self.coord_tower = ResidualTowerBlock(2, self.towers_hidden_dims, fusion_outdims)

        self.mega_tower = ResidualTowerBlock(19, 128, towers_outdims)

        self.image_tower = SplitHeadConvNeXt(
            pretrained=False,  # or False if training from scratch
            in_chans=4,  # Critical: override default 3-channel input
            outdims=towers_outdims,  # Your task's number of classes
        ).to(device)

        fusion_dims = 6 * towers_outdims + 3 * fusion_outdims
        # ===== Modality Fusion MoE =====
        # Combines features from all towers (4 metadata + image)
        self.fusion_experts = nn.ModuleList(
            [ResidualTowerBlock(fusion_dims, fusion_hidden_dims, 5) for _ in range(num_mlp_experts)]
        )

        num_experts = num_mlp_experts
        self.fusion_router = nn.Sequential(
            nn.Linear(fusion_dims, fusion_dims // 2),
            nn.Tanh(),
            nn.Dropout(0.3),
            nn.Linear(fusion_dims // 2, num_experts),
            # nn.Softmax(dim=-1)  # each expert gets independent [0,1] weight
            nn.Sigmoid(),
        )

    def forward(self, metadata, image):
        """Processes input metadata and optional image data through specialized towers,
        combines features using a Mixture of Experts (MoE) approach, and returns
        classification logits with expert weighting information.

        Parameters:
        -----------
        metadata : torch.Tensor
            Input metadata tensor of shape (batch_size, num_metadata_features).
            Expected to contain 24 metadata features (indices 0-23).
        image : torch.Tensor, optional
            Input image tensor of shape (batch_size, channels, height, width).
            If None, image features will be zero-initialized.

        Returns:
        --------
        dict
            Dictionary containing:
            - 'logits': torch.Tensor
                Output classification logits of shape (batch_size, num_classes)
            - 'expert_weights': torch.Tensor
                Raw fusion weights for all experts
            - 'fusion_weights': torch.Tensor
                Same as expert_weights (maintained for compatibility)"""

        # Process all metadata features through respective towers
        psf_feats = self.psf_tower(metadata[:, [5, 14]])  # PSF features
        lc_feats = self.lc_tower(metadata[:, [6, 9, 10, 13, 15, 17, 18, 19, 20, 21, 22, 23]])
        mag_feats = self.mag_tower(metadata[:, [6, 9, 10, 13, 15, 17, 18]])

        spatial_feats = self.spatial_tower(metadata[:, [2, 3, 4]])  # Spatial features
        nsta = self.nst1_tower(metadata[:, [0, 2]])  # Nearest source A features
        nstb = self.nst2_tower(metadata[:, [1, 3]])  # Nearest source B features
        coord_feats = self.coord_tower(metadata[:, [7, 8]])
        megatower = self.mega_tower(
            metadata[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]
        )

        # Process image if available (zeros otherwise)
        image_feats = self.image_tower(image) if image is not None else torch.zeros_like(nsta)

        # Concatenate all features for fusion
        all_feats = torch.cat(
            [nsta, nstb, spatial_feats, psf_feats, mag_feats, coord_feats, megatower, image_feats, lc_feats],
            dim=1,
        )

        # Fusion MoE - combine features from all modalities
        fusion_weights = self.fusion_router(all_feats)

        moe_output = torch.zeros(metadata.size(0), 5, device="cuda")  # ,device=metadata.to(device))

        topk_weights, topk_indices = torch.topk(fusion_weights, k=2, dim=-1)  # [B, k]

        # Process only through selected experts
        for expert_idx, expert in enumerate(self.fusion_experts):
            # Mask for samples where this expert is in top-k
            expert_mask = (topk_indices == expert_idx).any(dim=-1)  # [B]   # 'ResidualTowerBlock'

            if expert_mask.any():
                # Get weights for this expert [M] where M=sum(expert_mask)
                weights = topk_weights[expert_mask, (topk_indices[expert_mask] == expert_idx).nonzero()[:, 1]]
                # Compute expert output only for relevant samples
                expert_out = expert(all_feats[expert_mask])  # [M, num_classes]
                # Weighted contribution
                moe_output[expert_mask] += weights.unsqueeze(-1) * expert_out

        return moe_output

In [ ]:
# meta_model_path = '/projects/bcrv/abrown3/jobs/models_img_meta/best_model135_42dlletn.pth'
# model_obj = torch.load(meta_model_path, weights_only = False)
# model_obj.keys()

In [ ]:
XastroMiNN_model = XastroMiNN(
    num_classes=5,
    num_mlp_experts=meta_config["num_experts"],
    towers_hidden_dims=meta_config["towers_hidden_dims"],
    towers_outdims=meta_config["towers_outdims"],
    fusion_hidden_dims=meta_config["fusion_hidden_dims"],
    fusion_router_dims=meta_config["fusion_router_dims"],
    fusion_outdims=meta_config["fusion_outdims"],
    config=config,
).to(device)

XastroMiNN_model